In [ ]:
#| default_exp maze
from nbdev import *
from nbdev.showdoc import *

# Advent of Code Utils

> A collection of somewhat handy functions to make your AoC puzzle life solving a bit easier

In [2]:
#| exporti
from collections import namedtuple, deque
import heapq

## Maze algo's

In [ ]:
#| export
def bfs(connections, start, goal=None, verbose=False):
    """
    Requires a connections dict with tuples with neighbors per node.
    Or a connections function returning neighbors per node

    Returns
    if goal == None:    return dict of locations with neighbor closest to start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = deque([start]) # the locations that still need to be visited
    # paths = {start: [start]}
    isfunction = callable(connections)
    parents = {start: None}

    def get_path(parents,start,goal):
        # print(start,goals)
        cur = goal
        path = [cur]
        while cur != start:
            cur = parents[cur]
            path.append(cur)
        path.reverse()
        return path

    while frontier:
        search = frontier.popleft()
        if isfunction: neighbors = connections(search)
        else: neighbors = connections.get(search,None)
        if neighbors:
            for n in neighbors:
                if n not in seen:
                    seen.add(n)
                    frontier.append(n)
                    # paths[n] = paths[search]+[n]
                    parents[n]= search
                    if goal and n == goal:
                        # print('goal found')
                        return get_path(parents,start,goal)
                        # return paths[goal],parents
        seen.add(search)
    if goal: return False
    else: return parents

In [ ]:
def test_bfs(input):
    if input < 0: return (0,)
    elif input > 25: return (25,)
    else:
        return (input-1, input+1, input + 20, input -20)
bfs(test_bfs, 0,goal=10) == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

False

In [ ]:
#| export
def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = [(0,start)] # the locations that still need to be visited
    isfunction = callable(connections)
    parents = {start: (None,0)}

    def get_path(parents):
        cur = goal
        path = [cur]
        cost = parents[cur][1]
        while cur != start:
            cur = parents[cur][0]
            path.append(cur)
        path.reverse()
        return path,cost

    while frontier:
        # print('\n\n',frontier,'\n',parents)
        search_cost, search_node = heapq.heappop(frontier)
        # print('looking for', search_node,search_cost)
        if search_node == goal: break
        if isfunction: neighbors = connections(search_node)
        else: neighbors = connections.get(search_node,None)
        if neighbors:
            for n in neighbors:
                # print('n',n)
                if n[0] not in parents or n[1]+ search_cost < parents[n[0]][1]:
                    # print('updating')
                    heapq.heappush(frontier,(search_cost+n[1],n[0]))
                    # paths[n] = paths[search_node]+[n]
                    parents[n[0]]= (search_node,search_cost+n[1])
                        # return paths[goal],parents
        seen.add(search_node)
    if not goal: return parents
    elif goal in parents: return get_path(parents)
    else: return False


In [ ]:
test_dict = {1:[(2,1),(5,5)],
            2:[(1,1),(3,1)],
            3:[(2,1),(10,10)],
            5:[(1,1),(10,1)],
            10:[(3,1),(5,1)]
            }
assert dijkstra(test_dict, 1,goal=10) == ([1, 5, 10], 6)


In [ ]:
#| export
def get_path(parents,start,goal):
    # print(start,goals)
    cur = goal
    path = [cur]
    while cur != start:
        cur = parents[cur]
        path.append(cur)
    path.reverse()
    return path

In [ ]:
#| export
# found this on internet
def dfs(graph, start):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited